Putting it all together: A final benchmark and overview

In [ ]:
def avg(lst): 
    return sum(lst.timings) / len(lst.timings) 


Array-oriented programming(e.g NumPy) connects Python with compiled code, but it's not the only way to do that(like bindings, that we previously explored).

Although much faster than pure Python, array-oriented techniques are not as fast as imperative, compiled code.

In [ ]:
%%writefile quadratic_formula_c.c

#include <math.h>

void run(double* a, double* b, double* c, double* output) {
    for (int i = 0;  i < 1000000;  i++) {
        output[i] = (-b[i] + sqrt(b[i]*b[i] - 4*a[i]*c[i])) / (2*a[i]);
    }
}


In [ ]:
!cc quadratic_formula_c.c -shared -lm -o quadratic_formula_c.so

In [ ]:
import ctypes
import numpy as np

quadratic_formula_c = ctypes.CDLL("./quadratic_formula_c.so")
quadratic_formula_c.run.argtypes = (ctypes.POINTER(ctypes.c_double),) * 4
quadratic_formula_c.run.restype = None

In [ ]:
a = np.random.uniform(5, 10, 1000000)
b = np.random.uniform(10, 20, 1000000)
c = np.random.uniform(-0.1, 0.1, 1000000)

In [ ]:
output = np.zeros(1000000, dtype=np.float64)
quadratic_formula_c.run(*[arg.ctypes.data_as(ctypes.POINTER(ctypes.c_double)) for arg in (a, b, c, output)])
output

In [ ]:
ctypes_time = %timeit -o quadratic_formula_c.run(*[arg.ctypes.data_as(ctypes.POINTER(ctypes.c_double)) for arg in (a, b, c, output)])

In Pure Python it would look like:

In [ ]:
import math

def run(a, b, c):
    output = []
    for i in range(len(a)):
        output.append((-b[i] + math.sqrt(b[i]*b[i] - 4*a[i]*c[i])) / (2*a[i]))
    return output

In [ ]:
py_time = %timeit -o run(a, b, c)

Let's look at memory:

In [ ]:
%%memit

run(a, b, c)

NumPy is fast but not much better with memory:

In [ ]:
%load_ext memory_profiler

In [ ]:
%%memit

(-b + np.sqrt(b**2 - 4*a*c)) / (2*a)

Why? NumPy allocates a new array for each intermediate step.

* Memory allocation is expensive (`malloc` has to search for unused memory).
* Accessing different memory is expensive (the CPU can't re-use its cache, and acessing RAM is much slower than most mathematical operations).

To calculate the first expression, NumPy takes the steps shown in the second:

In [ ]:
tmp1 = np.negative(b)            # -b
tmp2 = np.square(b)              # b**2
tmp3 = np.multiply(4, a)         # 4*a
tmp4 = np.multiply(tmp3, c)      # tmp3*c
del tmp3
tmp5 = np.subtract(tmp2, tmp4)   # tmp2 - tmp4
del tmp2, tmp4
tmp6 = np.sqrt(tmp5)             # sqrt(tmp5)
del tmp5
tmp7 = np.add(tmp1, tmp6)        # tmp1 + tmp6
del tmp1, tmp6
tmp8 = np.multiply(2, a)         # 2*a
np.divide(tmp7, tmp8)            # tmp7 / tmp8     This is the result!

In [ ]:
np_time = %timeit -o (-b + np.sqrt(b**2 - 4*a*c)) / (2*a)

### Let's try Cython now

In [ ]:
%load_ext cython

In [ ]:
%%cython -a

from libc.math cimport sqrt

cpdef quadractic_cython(double[:] a, double[:] b, double[:] c, double[:] out):
        n = len(a)
        for i in range(n):
            out[i] = (-b[i] + sqrt(b[i]*b[i] - 4*a[i]*c[i])) / (2*a[i])

In [ ]:
cython_time = %timeit -o quadractic_cython(a, b, c, output)

In [ ]:
output

Best way to make C++ extensions for Python: pybind11

In [ ]:
%%writefile quadratic_formula_pybind11.cpp

#include <pybind11/pybind11.h>
#include <pybind11/numpy.h>
namespace py = pybind11;

void run(py::array_t<double, py::array::c_style | py::array::forcecast> a_numpy,
         py::array_t<double, py::array::c_style | py::array::forcecast> b_numpy,
         py::array_t<double, py::array::c_style | py::array::forcecast> c_numpy,
         py::array_t<double> output_numpy) {
    const double* a = a_numpy.data();
    const double* b = b_numpy.data();
    const double* c = c_numpy.data();
    double* output = output_numpy.mutable_data();
    for (int i = 0;  i < output_numpy.size();  i++) {
        output[i] = (-b[i] + sqrt(b[i]*b[i] - 4*a[i]*c[i])) / (2*a[i]);
    }
}

PYBIND11_MODULE(quadratic_formula_pybind11, m) {
    m.def("run", &run);
}

In [ ]:
import os
import sys
from pybind11 import get_include

inc = "-I " + get_include()
plat = "-undefined dynamic_lookup" if "darwin" in sys.platform else "-fPIC"
pyinc = !python3-config --cflags

In [ ]:
!c++ -std=c++11 quadratic_formula_pybind11.cpp -shared {inc} {pyinc.s} -o quadratic_formula_pybind11.so {plat}

In [ ]:
import quadratic_formula_pybind11

In [ ]:
output = np.zeros(1000000, dtype=np.float64)
quadratic_formula_pybind11.run(a, b, c, output)
output

In [ ]:
pybind_time = %timeit -o quadratic_formula_pybind11.run(a, b, c, output)

Leaving Python, writing C++ code, and then importing it* is fine for a long-term project, like a library that will be used many times, but it's inconvenient in the middle of a data analysis.

*If we change the C++, recompile, and do

```python
import quadratic_formula_pybind11
```

again, we will _not_ get the new version. We would still have the old version, with no error messages or warnings!

### JIT with LLVM: Numba

In [ ]:
import numba as nb

In [ ]:
@nb.njit
def quadratic_formula_numba(a, b, c):
    output = np.empty(len(a), dtype=np.float64)
    for i, (a_i, b_i, c_i) in enumerate(zip(a, b, c)):
        output[i] = (-b_i + np.sqrt(b_i**2 - 4*a_i*c_i)) / (2*a_i)
    return output

quadratic_formula_numba(a, b, c)

In [ ]:
nb_time = %timeit -o quadratic_formula_numba(a, b, c)

Bonus: JAX is a JIT compiler for _array-oriented_ Python.

In [ ]:
import jax
jax.config.update("jax_platform_name", "cpu")

In [ ]:
@jax.jit
def quadratic_formula_jax(a, b, c):
    return (-b + jax.numpy.sqrt(b**2 - 4*a*c)) / (2*a)

quadratic_formula_jax(a, b, c)

In [ ]:
jax_time = %timeit -o quadratic_formula_jax(a, b, c)

In [ ]:
avg(jax_time)

cppyy:

In [ ]:
import cppyy
cppyy.cppdef('''void run(double* a, double* b, double* c, double* output) {
    for (int i = 0;  i < 1000000;  i++) {
        output[i] = (-b[i] + sqrt(b[i]*b[i] - 4*a[i]*c[i])) / (2*a[i]);
    }
}
''')

In [ ]:
# double * from NumPy
x = a.ctypes.data_as(ctypes.POINTER(ctypes.c_double))
y = b.ctypes.data_as(ctypes.POINTER(ctypes.c_double))
z = c.ctypes.data_as(ctypes.POINTER(ctypes.c_double))
cppyy_out = output.ctypes.data_as(ctypes.POINTER(ctypes.c_double))

In [ ]:
cppyy_time = %timeit -o cppyy.gbl.run(x, y, z, cppyy_out)

In [ ]:
# NumPy from double *
np.ctypeslib.as_array(cppyy_out, shape = (1000000,))

In [ ]:
import matplotlib.pyplot as plt

test_names = [
    'Python "for" loops: imperative',
    'NumPy: array-oriented',
    'pybind11: imperative in C++',
    'Cython: imperative in a C++/Python hybrid',
    'Numba: imperative in a compiled subset of Python',
    "JAX: array-oriented with automatic fusion into JIT-compiled routines on CPU",
    "Cppyy: Automatic, C++ JIT with Clang"
]
test_results = np.array([
    avg(py_time),
    avg(np_time),
    avg(pybind_time),
    avg(cython_time),
    avg(nb_time),
    avg(jax_time),
    avg(cppyy_time)
])

# Creating the plot
plt.figure(figsize=(10, 6))
plt.barh(test_names, test_results, color='blue')
plt.xlabel('Average Execution Time (s)')
plt.ylabel('Benchmark Test')
plt.title('Benchmark Execution Time for Different Methods')
plt.gca().invert_yaxis()  # Invert y-axis to have the fastest method at the top
plt.show()